# Problem 969 - Kangaroo Hopping

Starting at zero, a kangaroo hops along the real number line in the positive direction. Each successive hop takes the kangaroo forward a uniformly random distance between $0$ and $1$. Let $H(n)$ be the expected number of hops needed for the kangaroo to pass $n$ on the real line.


If we write $\alpha = H(1)$, then for all positive integers $n$, $H(n)$ can be expressed as a polynomial function of $\alpha$ with rational coefficients. For example $H(3)=\alpha^3-2\alpha^2+\frac{1}{2}\alpha$. 

Define $S(n)$ to be the sum of all **integer** coefficients in this polynomial form of $H(n)$. Therefore $S(1)=1$ and $S(3)=1+(-2)=-1$.

You are also given $\displaystyle \sum_{n=1}^{10} S(n)=43$.

Find $\displaystyle\sum_{n=1}^{10^{18}} S(n)$. Give your answer modulo $10^9+7$.

## Solution.


In [4]:
from math import factorial
from tqdm import tqdm
from functools import cache

Using Irwin-Hull distribution we can derive a formula for H(n).

In [5]:
def H(n):
    return [(-1)**k * (n-k)**k * 1/factorial(k) for k in range(n+1)]

In [6]:
H(3)

[1.0, -2.0, 0.5, 0.0]

Thus, we need to find when $$(n-m)^m/m!$$ is an integer.


$$
m! \mid (n-m)^m \quad\Longleftrightarrow\quad n \equiv m \pmod{Q(m)},
$$

where  

$$
Q(m)=\prod_{\substack{p\ \text{prime}\\p\le m}} p,
$$

we obtain

$$
S(n)=
\sum_{\substack{0 \le m \le n-1 \\ n \equiv m \pmod{Q(m)}}}
(-1)^m \frac{(n-m)^m}{m!}.
$$


We want

$$
\Sigma(N)=\sum_{n=1}^{N} S(n).
$$

Insert the expression for \(S(n)\):

$$
\Sigma(N)=
\sum_{n=1}^{N}
\sum_{\substack{0\le m\le n-1\\ n\equiv m\pmod{Q(m)}}}
(-1)^m \frac{(n-m)^m}{m!}.
$$


Fix $m$.  
The condition $ n\equiv m\pmod{Q(m)} $ means

$$
n = m + t\,Q(m), \qquad t\ge 1.
$$

The constraint $ n\le N $ becomes

$$
1 \le t \le T_m,
\qquad
T_m = \left\lfloor \frac{N-m}{Q(m)} \right\rfloor.
$$

For such $ n $,

$$
(-1)^m \frac{(n-m)^m}{m!}
= (-1)^m \frac{(tQ(m))^m}{m!}
= (-1)^m \frac{Q(m)^m}{m!}\, t^m.
$$

Thus the contribution from this m is

$$
(-1)^m\frac{Q(m)^m}{m!}
\sum_{t=1}^{T_m} t^m.
$$
Finally 
$$
\Sigma(N)
=
\sum_{\substack{m\ge 0 \\ Q(m)\le N}}
(-1)^m \frac{Q(m)^m}{m!}
\left(
\sum_{t=1}^{T_m} t^m
\right),
$$

where

$$
T_m = \left\lfloor \frac{N-m}{Q(m)} \right\rfloor.
$$


In [70]:
from math import prod, factorial
from sympy import primerange

MOD = 10**9+7

def Q(m):
    if m == 0:
        return 1
    return prod(primerange(1, m+1))

def modinv(a, MOD):
    return pow(a, MOD-2, MOD)

def sum_powers(n, k, MOD):
    vals = [0]*(k+2)
    for i in range(k+2):
        vals[i] = sum(pow(t, k, MOD) for t in range(1, i+1)) % MOD
    
    def lagrange(x, y, n):
        k = len(y) - 1
        result = 0
        for i in range(k+1):
            num = 1
            den = 1
            for j in range(k+1):
                if i != j:
                    num = num * (n - j) % MOD
                    den = den * (i - j) % MOD
            result = (result + y[i] * num * modinv(den, MOD)) % MOD
        return result
    return lagrange(n, vals, n)


In [71]:
def sigma(N, MOD=MOD):
    ans = 0
    m = 0
    while True:
        qm = Q(m)
        if qm > N:
            break
        Tm = (N - m)//qm
        term = pow(qm, m, MOD) * modinv(factorial(m), MOD) % MOD
        term = term * sum_powers(Tm, m, MOD) % MOD
        if m % 2 == 1:
            term = MOD - term
        ans = (ans + term) % MOD
        m += 1
    return ans

In [72]:
sigma(10)

43

In [73]:
sigma(10**18)

412543690